In [1]:
# create entry points to spark
try:
    sc.stop()
except:
    pass
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
sc=SparkContext()
spark = SparkSession(sparkContext=sc)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/15 12:01:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Aggregate functions
Two aggregate functions:

* `aggregate()`
* `aggregateByKey()`

### `aggregate(zeroValue, seqOp, combOp)`

* **zeroValue** is like a data container. Its structure should match with the data structure of the returned values from the seqOp function.
* **seqOp** is a function that takes two arguments: the first argument is the zeroValue and the second argument is an element from the RDD. The zeroValue gets updated with the returned value after every run.
* **combOp** is a function that takes two arguments: the first argument is the final zeroValue from one partition and the other is another final zeroValue from another partition.

The code below calculates the total sum of squares for **mpg** and **disp** in data set **mtcars**.

Step 1: get some data.

In [2]:
mtcars_df = spark.read.csv('data/SparkData/mtcars.csv', inferSchema=True, header=True).select(['mpg', 'disp'])
mtcars_df.take(5)

[Row(mpg=21.0, disp=160.0),
 Row(mpg=21.0, disp=160.0),
 Row(mpg=22.8, disp=108.0),
 Row(mpg=21.4, disp=258.0),
 Row(mpg=18.7, disp=360.0)]

Step 2: calculate averages of mgp and disp

In [3]:
mpg_mean = mtcars_df.select('mpg').rdd.map(lambda x: x[0]).mean()
disp_mean = mtcars_df.select('disp').rdd.map(lambda x: x[0]).mean()
print('mpg mean = ', mpg_mean, '; ' 
      'disp mean = ', disp_mean)

mpg mean =  20.090625000000003 ; disp mean =  230.721875


Step 3: build **zeroValue, seqOp** and **combOp**

We are calculating two TSS. We create a tuple to store two values.

In [4]:
zeroValue = (0, 0) 

The **z** below refers to `zeroValue`. Its values get updated after every run. The **x** refers to an element in an RDD partition. In this case, both **z** and **x** have two values.

In [5]:
seqOp = lambda z, x: (z[0] + (x[0] - mpg_mean)**2, z[1] + (x[1] - disp_mean)**2)

The `combOp` function simply aggrate all `zeroValues` into one. 

In [6]:
combOp = lambda px, py: ( px[0] + py[0], px[1] + py[1] )

Implement `aggregate()` function.

In [7]:
mtcars_df.rdd.aggregate(zeroValue, seqOp, combOp)

(1126.0471874999998, 476184.7946875)

## `aggregateByKey(zeroValue, seqOp, combOp)`

This function does similar things as `aggregate()`. The `aggregate()` aggregate all results to the very end, but aggregateByKey() merge results by key.

### Let's first calculate word count, i.e., calculate/aggregate the counts based on the same key, i.e., word 

In [8]:
x=['hello', 'world', 'good', 'hello']

### Create an RDD, each element of the RDD to be a tuple, first element is key, which is word, second element is 1 to be counted.

In [ ]:
zz=sc.parallelize(x).map(lambda x: (x,1))

### define initial value, seqOp and combOp, since this is addition, initial value is 0

In [30]:
zero_value = 0
seqOp = (lambda x, y: x + y) #add up all the 1 based on the same key
combOp = (lambda x, y: x + y) #Merge all the word count across partitioning

In [31]:
zz.collect()

[('hello', 1), ('world', 1), ('good', 1), ('hello', 1)]

In [33]:
zz.aggregateByKey(zero_value,seqOp,combOp).collect()  #Now you have word count, counts based on words.

[('good', 1), ('hello', 2), ('world', 1)]

In [34]:
df=spark.read.csv('data/SparkData/mtcars.csv', inferSchema=True, header=True)

In [35]:
df.show(5)

23/04/15 12:15:54 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , mpg, cyl, disp, hp, drat, wt, qsec, vs, am, gear, carb
 Schema: _c0, mpg, cyl, disp, hp, drat, wt, qsec, vs, am, gear, carb
Expected: _c0 but found: 
CSV file: file:///Users/user/BerkeleySpark/data/SparkData/mtcars.csv
+-----------------+----+---+-----+---+----+-----+-----+---+---+----+----+
|              _c0| mpg|cyl| disp| hp|drat|   wt| qsec| vs| am|gear|carb|
+-----------------+----+---+-----+---+----+-----+-----+---+---+----+----+
|        Mazda RX4|21.0|  6|160.0|110| 3.9| 2.62|16.46|  0|  1|   4|   4|
|    Mazda RX4 Wag|21.0|  6|160.0|110| 3.9|2.875|17.02|  0|  1|   4|   4|
|       Datsun 710|22.8|  4|108.0| 93|3.85| 2.32|18.61|  1|  1|   4|   1|
|   Hornet 4 Drive|21.4|  6|258.0|110|3.08|3.215|19.44|  1|  0|   3|   1|
|Hornet Sportabout|18.7|  8|360.0|175|3.15| 3.44|17.02|  0|  0|   3|   2|
+-----------------+----+---+-----+---+----+-----+-----+---+---+----+----+
only showing top 5 r

### Create a SparkSQL dataframe that contains on 3 columns, cyl (cylinder, mpg and disp (displacement))

In [36]:
car_df=df.select(['cyl', 'mpg', 'disp'])

In [37]:
car_df.show(5)

+---+----+-----+
|cyl| mpg| disp|
+---+----+-----+
|  6|21.0|160.0|
|  6|21.0|160.0|
|  4|22.8|108.0|
|  6|21.4|258.0|
|  8|18.7|360.0|
+---+----+-----+
only showing top 5 rows



### Create another dataframe that contains cylinder, mean mpg and mean displacement

In [38]:
car_df.createOrReplaceTempView('car')

In [39]:
car_mean_df=spark.sql("select cyl as cylinder, avg(mpg) as mean_mpg, avg(disp) as mean_disp from car group by 1")

In [40]:
car_mean_df.show()

+--------+------------------+------------------+
|cylinder|          mean_mpg|         mean_disp|
+--------+------------------+------------------+
|       6| 19.74285714285714|183.31428571428572|
|       4|26.663636363636364|105.13636363636364|
|       8|15.100000000000003|353.09999999999997|
+--------+------------------+------------------+



In [41]:
car_df=car_df.withColumnRenamed("cyl","cylinder") #rename the column cyl of car_df to cylinder
car_df.show(5)
car_mean_df.show()

+--------+----+-----+
|cylinder| mpg| disp|
+--------+----+-----+
|       6|21.0|160.0|
|       6|21.0|160.0|
|       4|22.8|108.0|
|       6|21.4|258.0|
|       8|18.7|360.0|
+--------+----+-----+
only showing top 5 rows

+--------+------------------+------------------+
|cylinder|          mean_mpg|         mean_disp|
+--------+------------------+------------------+
|       6| 19.74285714285714|183.31428571428572|
|       4|26.663636363636364|105.13636363636364|
|       8|15.100000000000003|353.09999999999997|
+--------+------------------+------------------+



### Create a new dataframe that combines mpg, average mpg, displacement, average displacement along with cylinder

In [42]:
car_df.createOrReplaceTempView("car_df")
car_mean_df.createOrReplaceTempView("car_mean_df")
car_mean_combined_df=spark.sql("select a.cylinder, a.mpg, b.mean_mpg, a.disp, b.mean_disp from car_df a inner join \
           car_mean_df b on a.cylinder = b.cylinder")
car_mean_combined_df.show(5)





+--------+----+------------------+-----+------------------+
|cylinder| mpg|          mean_mpg| disp|         mean_disp|
+--------+----+------------------+-----+------------------+
|       6|21.0| 19.74285714285714|160.0|183.31428571428572|
|       6|21.0| 19.74285714285714|160.0|183.31428571428572|
|       4|22.8|26.663636363636364|108.0|105.13636363636364|
|       6|21.4| 19.74285714285714|258.0|183.31428571428572|
|       8|18.7|15.100000000000003|360.0|353.09999999999997|
+--------+----+------------------+-----+------------------+
only showing top 5 rows



### Now create a RDD, each element of RDD is a list, 1st element of the list is always Key, Cylinder, 2nd element of the list is value, a tuple 

[cylinder, (mpg, mean_mpg, disp, mean_disp) ]

In [43]:
car_mean_combined_rdd=car_mean_combined_df.rdd.map(lambda x: [x[0], (x[1], x[2], x[3], x[4])])

In [44]:
car_mean_combined_rdd.take(4)
    
    

[[6, (21.0, 19.74285714285714, 160.0, 183.31428571428572)],
 [6, (21.0, 19.74285714285714, 160.0, 183.31428571428572)],
 [4, (22.8, 26.663636363636364, 108.0, 105.13636363636364)],
 [6, (21.4, 19.74285714285714, 258.0, 183.31428571428572)]]

### Create a new RDD that contains key and value:

[cynlinder, (mean suare error of mpg, mean square error of disp)]


In [45]:
car_mean_square_rdd=car_mean_combined_rdd.map(lambda x: [x[0], ((x[1][0]-x[1][1])**2, (x[1][2]-x[1][3])**2)])
car_mean_square_rdd.take(5)

[[6, (1.5804081632653129, 543.5559183673471)],
 [6, (1.5804081632653129, 543.5559183673471)],
 [4, (14.92768595041322, 8.200413223140474)],
 [6, (2.746122448979596, 5577.955918367346)],
 [8, (12.959999999999972, 47.61000000000047)]]

Let's compute the squqre mean error between sum((mpg-mean_mpg)^2) and sum((disp-mean_disp)^) based on same key
which is cylinder, and use aggregateByKey API

In [47]:
zero_value = (0, 0) #zero_value[0] is for mpg, zero_value[1] is for disp
seqOp = (lambda x, y: (x[0]+y[0], x[1]+y[1])) #add up all mean square error of mpg, add up all mean square error disp
combOp = (lambda x, y: (x[0] + y[0], x[1] + y[1])) #merge all mean square errors across partitions

In [48]:
car_mean_square_rdd.aggregateByKey(zero_value, seqOp, combOp).collect()

[(6, (12.677142857142847, 10364.628571428573)),
 (4, (203.38545454545448, 7220.825454545454)),
 (8, (85.19999999999999, 59708.38))]